# Level 2 - Week 6 - 01 Agent Loop and State

**Estimated time:** 60-90 minutes

## Learning Objectives

- Track explicit state per step
- Store tool inputs and outputs
- Define stop conditions


## Overview

Agents are loops with explicit state.
State makes debugging possible.

## Practice Steps

- Define Step and AgentState dataclasses.
- Implement run_agent with step cap.


### Sample code

State model for agent steps.


In [ ]:
from dataclasses import dataclass, field
from typing import Dict, Any, List

@dataclass
class Step:
    tool: str
    tool_input: Dict[str, Any]
    tool_output: Dict[str, Any] | None = None
    error: str | None = None

@dataclass
class AgentState:
    task: str
    plan: List[str] = field(default_factory=list)
    steps: List[Step] = field(default_factory=list)
    final: str | None = None


### Student fill-in

Implement run_agent and store steps.


In [ ]:
def run_agent(task: str, max_steps: int = 3) -> AgentState:
    state = AgentState(task=task, plan=['search', 'write_answer'])
    # TODO: implement tool calls and stop conditions
    return state


## Self-check

- Is every tool call stored in steps?
- Do you cap max_steps?


Legacy practice content from practice.ipynb

# Level 2 — Week 6 Practice: Agent Foundations

**Estimated time:** 60–90 minutes

## Learning Objectives

- Build a minimal plan → tool → observe loop
- Track state across steps
- Add stop conditions and step caps
- Keep tool interfaces explicit


Legacy practice content from practice.ipynb

## Overview

Agents are just structured loops with tools and state. You will implement a
minimal runner that is deterministic and testable.

You will:

1. Define a tool interface.
2. Run a loop with a step cap.
3. Add a simple stop condition.

## Practice Steps

- Implement a toy tool.
- Run the loop and inspect state.
- Add a stop condition based on output.


In [ ]:
# Legacy practice content
TASK_6_1_GUIDE = """
Task 6.1: Agent loop

Fill in the TODOs to make the loop deterministic and stoppable.

Checklist:
- Use a fixed tool order
- Enforce max_steps
- Stop when results are present
"""

print(TASK_6_1_GUIDE)


In [ ]:
# Legacy practice content
from dataclasses import dataclass
from typing import Any, Dict, Callable, List

@dataclass
class Tool:
    name: str
    fn: Callable[[Dict[str, Any]], Dict[str, Any]]

def search_tool(payload: Dict[str, Any]) -> Dict[str, Any]:
    return {"results": [f"hit for {payload['query']}"]}

def run_agent(task: str, tools: List[Tool], max_steps: int = 3) -> Dict[str, Any]:
    state: Dict[str, Any] = {"task": task, "steps": []}
    for step in range(max_steps):
        # TODO: replace with model planning
        tool = tools[0]
        tool_input = {"query": task}
        tool_output = tool.fn(tool_input)
        state["steps"].append({"tool": tool.name, "input": tool_input, "output": tool_output})
        # TODO: replace with smarter stop condition
        if tool_output.get("results"):
            break
    state["result"] = state["steps"][-1]["output"] if state["steps"] else {}
    return state

agent = run_agent("find refund policy", [Tool(name="search", fn=search_tool)])
print(agent)


Legacy practice content from practice.ipynb

## Self-check

- Is the loop deterministic?
- Are stop conditions explicit?
- Is state tracked clearly?
